In [52]:
import quandl
quandl.ApiConfig.api_key = "KxKMd8dsHx1sWnaGj6_U"
import numpy as np
import matplotlib as plt
import pandas as pd
import keras
from sklearn.preprocessing import MinMaxScaler

In [53]:
def get_monthly_close_prices_ticker(ticker,start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SEP',ticker=ticker,date={'gte':start_date,'lte':end_date}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)
    close = df.pivot(index='date', columns='ticker', values='close')
    monthly = close.resample('M').apply(lambda x: x[-1])
    #.agg(lambda x: x[-1])
    return monthly

In [54]:
monthly_prices = get_monthly_close_prices_ticker('A','2017-13-31','2019-01-01')

In [55]:
monthly_prices

ticker,A
date,
2018-01-31,73.43
2018-02-28,68.59
2018-03-31,66.90
2018-04-30,65.74
2018-05-31,61.92
2018-06-30,61.84
2018-07-31,66.04
2018-08-31,67.54
2018-09-30,70.54


In [56]:
#quandl.get_table('SHARADAR/SEP',date={'gte':'2018-06-30','lte':'2019-01-01'}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)

In [57]:
def get_monthly_returns(monthly_prices):
    """
    Compute returns for each ticker and date in close.
    
    Parameters
    ----------
    close : DataFrame
        Close prices for each ticker and date
    
    Returns
    -------
    returns : DataFrame
        Returns for each ticker and date
    """
        

    #.agg(lambda x: x[-1])
    monthly_returns = (monthly_prices - monthly_prices.shift(1))/monthly_prices.shift(1).fillna(value=0)
    
    
    return monthly_returns

In [58]:
df1 = get_monthly_returns(monthly_prices)


In [59]:
df1.head()

ticker,A
date,
2018-01-31,NaN
2018-02-28,-0.065913
2018-03-31,-0.024639
2018-04-30,-0.017339
2018-05-31,-0.058108


In [60]:
df1 = df1.iloc[ 1: , : ]

In [61]:
df1

ticker,A
date,
2018-02-28,-0.065913
2018-03-31,-0.024639
2018-04-30,-0.017339
2018-05-31,-0.058108
2018-06-30,-0.001292
2018-07-31,0.067917
2018-08-31,0.022714
2018-09-30,0.044418
2018-10-31,-0.081514


In [62]:
df1.reset_index(inplace=True)


In [63]:
df1.head()

ticker,date,A
0,2018-02-28,-0.065913
1,2018-03-31,-0.024639
2,2018-04-30,-0.017339
3,2018-05-31,-0.058108
4,2018-06-30,-0.001292


In [69]:
def get_factor_data_ticker(ticker,start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SF1',ticker=ticker,dimension='ARQ',calendardate={'gte':start_date, 'lte':end_date}, paginate=True).fillna(value=0)

    
    return df

In [70]:
df2 = get_factor_data_ticker('A','2017-13-31','2019-01-01')

In [71]:
df2.head()

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
0,A,ARQ,2018-12-31,2019-03-05,2019-01-31,2019-03-05,-381000000,8952000000,0,3712000000,...,317515869,318000000,322000000,4.038,5253000000,0,-256000000,0,16.519,2617000000
1,A,ARQ,2018-09-30,2018-12-20,2018-10-31,2019-03-05,-408000000,8541000000,0,3848000000,...,318533054,318000000,322000000,4.069,5077000000,0,49000000,0,15.965,2677000000
2,A,ARQ,2018-06-30,2018-08-30,2018-07-31,2019-03-05,-335000000,8349000000,0,3667000000,...,318769547,320000000,324000000,3.759,4901000000,0,6000000,0,15.316,2653000000
3,A,ARQ,2018-03-31,2018-05-31,2018-04-30,2019-03-05,-310000000,8784000000,0,4525000000,...,319952126,322000000,326000000,3.745,5852000000,0,22000000,0,18.174,3160000000


In [72]:
df2['calendardate'] = pd.to_datetime(df2['calendardate']).dt.strftime("%Y-%m-%d")


In [73]:
def add_returns(row, df):
    c_date = row['calendardate']
    ticker = row['ticker']
    result = 0.0
    value = df.loc[df['date'] == c_date]
    if not value.empty:
        if ticker in value.columns:
            result = value[ticker].values[0]
    return result
df2['results'] = df2.apply(lambda row: add_returns(row, df1), axis=1)

In [74]:
df2.sort_values('calendardate')

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,results
None,,,,,,,,,,,,,,,,,,,,,
3,A,ARQ,2018-03-31,2018-05-31,2018-04-30,2019-03-05,-310000000,8784000000,0,4525000000,...,322000000,326000000,3.745,5852000000,0,22000000,0,18.174,3160000000,-0.024639
2,A,ARQ,2018-06-30,2018-08-30,2018-07-31,2019-03-05,-335000000,8349000000,0,3667000000,...,320000000,324000000,3.759,4901000000,0,6000000,0,15.316,2653000000,-0.001292
1,A,ARQ,2018-09-30,2018-12-20,2018-10-31,2019-03-05,-408000000,8541000000,0,3848000000,...,318000000,322000000,4.069,5077000000,0,49000000,0,15.965,2677000000,0.044418
0,A,ARQ,2018-12-31,2019-03-05,2019-01-31,2019-03-05,-381000000,8952000000,0,3712000000,...,318000000,322000000,4.038,5253000000,0,-256000000,0,16.519,2617000000,-0.067588


In [23]:
#corr_matrix = df.corr().abs()

In [24]:
#upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [25]:
#to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [27]:
#df2 = df2.drop(df2[to_drop], axis=1)

In [28]:
#df2.shape

(8412, 87)

In [85]:
dataset = df2.set_index('calendardate')
dataset



,ticker,dimension,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,results
calendardate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-12-31,A,ARQ,2019-03-05,2019-01-31,2019-03-05,-381000000,8952000000,0,3712000000,5240000000,0,15.836,-39000000,2057000000,2057000000,577000000,504000000,3.390,0.778,1798000000,0,1798000000,1798000000,346000000,54000000,0,0.008,0.164,266000000,320000000,0.249,320000000,266000000,248000000,1.58,1.57,1.58,5036000000,0,5036000000,25183546583,24,20.195,174000000,0.547,1.0,707000000,0.551,3699000000,18000000,3899000000,0,653000000,77000000,0,77000000,3916000000,1095000000,2821000000,25442546583,-190000000,-248000000,-53000000,0,-52000000,-122000000,-290000000,-1000000,213000000,9000000,504000000,504000000,504000000,0,0,0.393,457000000,250000000,315000000,0.104,5.052,22.318,22.508,829000000,0,80.13,5.102,5.110,833000000,90000000,1284000000,1284000000,102000000,0,0,0,0,24000000,355000000,1.0,317515869,318000000,322000000,4.038,5253000000,0,-256000000,0,16.519,2617000000,-0.067588
2018-09-30,A,ARQ,2018-12-20,2018-10-31,2019-03-05,-408000000,8541000000,0,3848000000,4693000000,0,14.362,-35000000,2247000000,2247000000,585000000,195000000,3.286,0.869,1799000000,0,1799000000,1799000000,324000000,56000000,0,0.009,0.149,262000000,318000000,0.246,318000000,262000000,244000000,0.60,0.60,0.60,4567000000,0,4567000000,20317169790,20,16.505,337000000,1.060,1.0,709000000,0.548,3464000000,18000000,3458000000,0,638000000,68000000,0,68000000,3970000000,1171000000,2799000000,20765169790,-432000000,-79000000,-83000000,0,-47000000,-131000000,-114000000,1000000,372000000,-11000000,195000000,195000000,195000000,0,0,0.151,460000000,249000000,340000000,0.248,4.547,65.713,66.520,822000000,0,65.19,4.226,4.258,776000000,-336000000,1294000000,1294000000,104000000,0,0,0,0,14000000,356000000,1.0,318533054,318000000,322000000,4.069,5077000000,0,49000000,0,15.965,2677000000,0.044418
2018-06-30,A,ARQ,2018-08-30,2018-07-31,2019-03-05,-335000000,8349000000,0,3667000000,4682000000,0,14.262,-33000000,2131000000,2131000000,542000000,236000000,3.616,0.828,1799000000,0,1799000000,1799000000,328000000,53000000,0,0.009,0.149,260000000,313000000,0.260,313000000,260000000,242000000,0.74,0.73,0.74,4564000000,0,4564000000,21108439731,21,17.431,164000000,0.512,1.0,661000000,0.549,3448000000,18000000,3555000000,0,623000000,70000000,0,70000000,3781000000,1014000000,2767000000,21440439731,0,-430000000,-226000000,-315000000,-48000000,-589000000,-473000000,-10000000,197000000,-15000000,236000000,236000000,236000000,0,0,0.196,436000000,225000000,273000000,0.201,4.698,71.948,71.553,801000000,0,67.26,4.458,4.476,733000000,-416000000,1203000000,1203000000,97000000,0,0,0,0,13000000,339000000,1.0,318769547,320000000,324000000,3.759,4901000000,0,6000000,0,15.316,2653000000,-0.001292
2018-03-31,A,ARQ,2018-05-31,2018-04-30,2019-03-05,-310000000,8784000000,0,4525000000,4259000000,0,14.326,-48000000,3011000000,3011000000,562000000,205000000,3.315,0.903,2115000000,315000000,1800000000,2115000000,333000000,50000000,0,0.009,0.149,246000000,296000000,0.245,296000000,246000000,227000000,0.64,0.63,0

In [86]:
df_numbers = dataset.drop(columns=['ticker','dimension','reportperiod','lastupdated','datekey'])

In [87]:
pd.options.display.max_columns = None
display(df_numbers)

,datekey,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,results
calendardate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-12-31,2019-03-05,-381000000,8952000000,0,3712000000,5240000000,0,15.836,-39000000,2057000000,2057000000,577000000,504000000,3.390,0.778,1798000000,0,1798000000,1798000000,346000000,54000000,0,0.008,0.164,266000000,320000000,0.249,320000000,266000000,248000000,1.58,1.57,1.58,5036000000,0,5036000000,25183546583,24,20.195,174000000,0.547,1.0,707000000,0.551,3699000000,18000000,3899000000,0,653000000,77000000,0,77000000,3916000000,1095000000,2821000000,25442546583,-190000000,-248000000,-53000000,0,-52000000,-122000000,-290000000,-1000000,213000000,9000000,504000000,504000000,504000000,0,0,0.393,457000000,250000000,315000000,0.104,5.052,22.318,22.508,829000000,0,80.13,5.102,5.110,833000000,90000000,1284000000,1284000000,102000000,0,0,0,0,24000000,355000000,1.0,317515869,318000000,322000000,4.038,5253000000,0,-256000000,0,16.519,2617000000,-0.067588
2018-09-30,2018-12-20,-408000000,8541000000,0,3848000000,4693000000,0,14.362,-35000000,2247000000,2247000000,585000000,195000000,3.286,0.869,1799000000,0,1799000000,1799000000,324000000,56000000,0,0.009,0.149,262000000,318000000,0.246,318000000,262000000,244000000,0.60,0.60,0.60,4567000000,0,4567000000,20317169790,20,16.505,337000000,1.060,1.0,709000000,0.548,3464000000,18000000,3458000000,0,638000000,68000000,0,68000000,3970000000,1171000000,2799000000,20765169790,-432000000,-79000000,-83000000,0,-47000000,-131000000,-114000000,1000000,372000000,-11000000,195000000,195000000,195000000,0,0,0.151,460000000,249000000,340000000,0.248,4.547,65.713,66.520,822000000,0,65.19,4.226,4.258,776000000,-336000000,1294000000,1294000000,104000000,0,0,0,0,14000000,356000000,1.0,318533054,318000000,322000000,4.069,5077000000,0,49000000,0,15.965,2677000000,0.044418
2018-06-30,2018-08-30,-335000000,8349000000,0,3667000000,4682000000,0,14.262,-33000000,2131000000,2131000000,542000000,236000000,3.616,0.828,1799000000,0,1799000000,1799000000,328000000,53000000,0,0.009,0.149,260000000,313000000,0.260,313000000,260000000,242000000,0.74,0.73,0.74,4564000000,0,4564000000,21108439731,21,17.431,164000000,0.512,1.0,661000000,0.549,3448000000,18000000,3555000000,0,623000000,70000000,0,70000000,3781000000,1014000000,2767000000,21440439731,0,-430000000,-226000000,-315000000,-48000000,-589000000,-473000000,-10000000,197000000,-15000000,236000000,236000000,236000000,0,0,0.196,436000000,225000000,273000000,0.201,4.698,71.948,71.553,801000000,0,67.26,4.458,4.476,733000000,-416000000,1203000000,1203000000,97000000,0,0,0,0,13000000,339000000,1.0,318769547,320000000,324000000,3.759,4901000000,0,6000000,0,15.316,2653000000,-0.001292
2018-03-31,2018-05-31,-310000000,8784000000,0,4525000000,4259000000,0,14.326,-48000000,3011000000,3011000000,562000000,205000000,3.315,0.903,2115000000,315000000,1800000000,2115000000,333000000,50000000,0,0.009,0.149,246000000,296000000,0.245,296000000,246000000,227000000,0.64,0.63,0.64,4613000000,0,4613000000,18915435642,20,16.292,255000000,0.792,1.0,644000000,0.534,2932000000,19000000,3591000000,0,594000000,139000000,0,139000000,4167000

In [88]:
df_numbers = dataset.drop(columns=('datekey'))

In [92]:
df_numbers = dataset.drop(columns=['ticker','dimension','reportperiod','lastupdated','datekey'])

In [103]:
dataset = df_numbers.values
dataset

array([[-3.81000000e+08,  8.95200000e+09,  0.00000000e+00,
         3.71200000e+09,  5.24000000e+09,  0.00000000e+00,
         1.58360000e+01, -3.90000000e+07,  2.05700000e+09,
         2.05700000e+09,  5.77000000e+08,  5.04000000e+08,
         3.39000000e+00,  7.78000000e-01,  1.79800000e+09,
         0.00000000e+00,  1.79800000e+09,  1.79800000e+09,
         3.46000000e+08,  5.40000000e+07,  0.00000000e+00,
         8.00000000e-03,  1.64000000e-01,  2.66000000e+08,
         3.20000000e+08,  2.49000000e-01,  3.20000000e+08,
         2.66000000e+08,  2.48000000e+08,  1.58000000e+00,
         1.57000000e+00,  1.58000000e+00,  5.03600000e+09,
         0.00000000e+00,  5.03600000e+09,  2.51835466e+10,
         2.40000000e+01,  2.01950000e+01,  1.74000000e+08,
         5.47000000e-01,  1.00000000e+00,  7.07000000e+08,
         5.51000000e-01,  3.69900000e+09,  1.80000000e+07,
         3.89900000e+09,  0.00000000e+00,  6.53000000e+08,
         7.70000000e+07,  0.00000000e+00,  7.70000000e+0

In [104]:
train = dataset[0:3]
valid = df_numbers[3:]

In [100]:
x_train = np.reshape(x_train.shape[0],x_train.shape[1],1) 

ValueError: cannot reshape array of size 1 into shape (106,)

In [ ]:
from sklearn.preprocessing import scale

# Remove unimportant features (weekdays)
train_features = train_features.iloc[:, :-4]
test_features = test_features.iloc[:, :-4]

# Standardize the train and test features
scaled_train_features = scale(train_features)
scaled_test_features = scale(test_features)

# Plot histograms of the 14-day SMA RSI before and after scaling
f, ax = plt.subplots(nrows=2, ncols=1)
train_features.iloc[:, 2].hist(ax=ax[0])
ax[1].hist(scaled_train_features[:, 2])
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Create the model
model_1 = Sequential()
model_1.add(Dense(100, input_dim=scaled_train_features.shape[1], activation='relu'))
model_1.add(Dense(20, activation='relu'))
model_1.add(Dense(1, activation='linear'))

# Fit the model
model_1.compile(optimizer='adam', loss='mse')
history = model_1.fit(scaled_train_features, train_targets, epochs=25)